## Import Statements

In [ ]:
import struct
import cv2 as cv
import numpy as np
import torch.nn as nn
from sklearn.svm import SVC
from google.colab import drive
from skimage.feature import hog
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from skimage.morphology import erosion
from torchvision.datasets import MNIST
from skimage.transform import rescale, resize
from sklearn.metrics import classification_report

drive.mount("/content/drive")

## Question 2

In [ ]:
# Reading the image
img = cv.imread('faces.png')

converted_img = cv.cvtColor(img, cv.COLOR_BGR2RGB)
  
# Converting image to grayscale
gray_img = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
  
# Loading the required haar-cascade xml classifier file (the classifier has already been trained with ADABoost Algorithm )
haar_cascade = cv.CascadeClassifier(cv.data.haarcascades + 'haarcascade_frontalface_default.xml')
  
# Applying the face detection method on the grayscale image
faces_rect = haar_cascade.detectMultiScale(gray_img, 1.1, 9)
  
# Iterating through rectangles of detected faces
for (x, y, w, h) in faces_rect:
    cv.rectangle(converted_img, (x, y), (x+w, y+h), (0, 255, 0), 2)

plt.figure(figsize = (20, 10))
plt.imshow(converted_img)
plt.show()

## Question 3

In [1]:
# read the grayscale image
img = cv.imread("/content/tounching_grayscale.png", cv.IMREAD_GRAYSCALE)

# use OTSU thresholding to binarize the image
(thresh, binary_img) = cv.threshold(img, 100, 255, cv.THRESH_BINARY | cv.THRESH_OTSU)

# plot the images
fig, ax = plt.subplots(1, 2, figsize = (15, 7))
ax[0].imshow(img, cmap = "gray")
ax[0].set_title("Original Image")
ax[1].set_title("Binary Image")
ax[1].imshow(binary_img, cmap = "gray")
plt.show()

NameError: ignored

In [ ]:
# convert the image into a feature map and apply KMeans algorithm to get the cluster centroids 

X = np.asarray(np.where(binary_img == 255)).T
kmeans_model = KMeans(n_clusters = 12, max_iter = 300)
kmeans_model.fit(X)

plt.figure(figsize = (10, 7))

# plot the centroids
f1 = np.round(kmeans_model.cluster_centers_.T[1]).astype(np.int16)
f2 = np.round(kmeans_model.cluster_centers_.T[0]).astype(np.int16)

ax = plt.gca()
X_f = np.asarray([f1, f2])
plt.scatter(f1, f2)
plt.imshow(binary_img, cmap = "gray")
plt.title("Centroids for each cluster found by K-Means")
plt.show()

In [ ]:
# set the number iterations for eroding an image
erosion_level = 1

# define a blank final image
final_img = np.zeros_like(binary_img)

# add each eroded cluster
for i in range(12):

  # get the labels for each point, corresponding to each predicted cluster
  labels_index = X[np.where(kmeans_model.labels_ == i)[0]]

  # define a blank image
  blank_image = np.zeros_like(binary_img)
  
  # plot all the points of each cluster on the blank image
  for j in labels_index:
    blank_image[j[0], j[1]] = 255
  
  # erode the blank image to prevent it touching from other clusters
  for k in range(erosion_level):
    blank_image = erosion(blank_image)
  
  # add the cluster to the final image
  final_img += blank_image

# plot both the images
fig, ax = plt.subplots(1, 2, figsize = (15, 7))
ax[0].imshow(binary_img, cmap = "gray")
ax[0].set_title("Before Clustering and Seperating")
ax[1].set_title("After Clustering and Seperating")
ax[1].imshow(final_img, cmap = "gray")
plt.show()

## Question 4

In [ ]:
# read the image
og_img = cv.imread("/content/shapes.png",cv.IMREAD_GRAYSCALE)
# convert the image into a binary image
(thresh, binary_img) = cv.threshold(og_img, 100, 1, cv.THRESH_BINARY)
# plot the image
plt.figure(figsize = (8, 8))
plt.imshow(binary_img, cmap = "gray")
plt.title("Original Image")
plt.show()

In [ ]:
# get a copy of the image and draw contours on the copy
edged = binary_img.copy()
contours, hierarchy = cv.findContours(edged, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_NONE)

# plot the required figures
nrows = 1
ncols = 3

fig, ax = plt.subplots(nrows, ncols, figsize = (25, 7))

for row in range(nrows):
  for col in range(ncols):

    contour_no = row*ncols + col
    
    # fit ellipse
    (x,y),(MA,ma),angle = cv.fitEllipse(contours[contour_no])
    im = cv.cvtColor(binary_img, cv.COLOR_GRAY2RGB)
    
    # get the eccentricity
    e = np.sqrt(abs((MA/2)**2 - (ma/2)**2))/max(MA/2, ma/2)
    
    # get the bounding Rectangle
    x,y,w,h = cv.boundingRect(contours[contour_no])
    
    # show all
    ax[col].imshow(cv.rectangle(im,(x,y),(x+w,y+h),(0,255,0),1))
    ax[col].imshow(binary_img, cmap = "gray", alpha = 0.3)
    ax[col].set_title(f"e={np.round(e, 3)}")
    
plt.suptitle("Bounding Box and Eccentricity(e) for all objects")
plt.show()

## Question 5


In [ ]:
c1 = cv.imread("d1.png")
c2 = cv.imread("d2.png")

c1 = cv.cvtColor(c1, cv.COLOR_BGR2RGB)
c2 = cv.cvtColor(c2, cv.COLOR_BGR2RGB)

fig, ax = plt.subplots(1, 2, figsize = (20, 10))
ax[0].imshow(c1)
ax[1].imshow(c2)
plt.show()

### Harris Corner Detector

In [ ]:
# convert to grayscale
gray_c1 = cv.cvtColor(c1, cv.COLOR_RGB2GRAY)
gray_c2 = cv.cvtColor(c2, cv.COLOR_RGB2GRAY)

# detect with Harris corner detector, and dilate for more visibility
c1_dst = cv.cornerHarris(gray_c1,2,3,0.1)
c1_dst = cv.dilate(c1_dst, None)

c2_dst = cv.cornerHarris(gray_c2,2,3,0.1)
c2_dst = cv.dilate(c2_dst, None)

# create images
marker_img_c1 = c1.copy()
marker_img_c1[c1_dst > 0.05*np.max(c1_dst)] = [255,0,0]

marker_img_c2 = c2.copy()
marker_img_c2[c2_dst > 0.05*np.max(c2_dst)] = [255,0,0]

fig, ax = plt.subplots(1, 2, figsize = (20, 10))
ax[0].imshow(marker_img_c1)
ax[1].imshow(marker_img_c2)
plt.show()

### ORB detector

In [ ]:
# Initialize the ORB detector algorithm
orb = cv.ORB_create()
  
# detect keypoints and descriptors
queryKeypoints, queryDescriptors = orb.detectAndCompute(gray_c2,None)
trainKeypoints, trainDescriptors = orb.detectAndCompute(gray_c1,None)
 
# Initialize the Matcher for matching
# the keypoints and then match the
# keypoints
matcher = cv.BFMatcher()
matches = matcher.match(queryDescriptors,trainDescriptors)

final_img = cv.drawMatches(c1, queryKeypoints,
c2, trainKeypoints, matches[:20],None)
  
final_img = cv.resize(final_img, (1000,500))
 
# Show the final image
plt.figure(figsize = (20, 10))
plt.imshow(final_img)
plt.show()

## Question 6

In [ ]:
# get the dataset
fake_images = []
real_images = []

for file in os.listdir("/kaggle/input/real-and-fake-face-detection/real_and_fake_face/training_fake"):
    filename = r"../input/real-and-fake-face-detection/real_and_fake_face/training_fake/" + file
    fake_images.append(cv.imread(filename, cv.IMREAD_GRAYSCALE))
for file in os.listdir("/kaggle/input/real-and-fake-face-detection/real_and_fake_face/training_real"):
    filename = r"../input/real-and-fake-face-detection/real_and_fake_face/training_real/" + file
    real_images.append(cv.imread(filename, cv.IMREAD_GRAYSCALE))    

In [ ]:
# get HOG features based on the following parameters
def get_hog_features(j):
    return hog(resize(j, (128,64)), cells_per_block = (8,8), pixels_per_cell = (8, 8))


# iterate over each image and get hog features for each image

X = np.asarray(real_images + fake_images)
y = np.asarray([1] * len(real_images) + [0] * len(fake_images))

train_data, test_data, train_labels, test_labels = train_test_split(X, y, test_size = 0.25, shuffle = True, stratify = y)

hog_train_data = []
hog_test_data = []

for i in range(train_data.shape[0]):
    hog_train_data.append(get_hog_features(train_data[i,:,:]))

for i in range(test_data.shape[0]):
    hog_test_data.append(get_hog_features(test_data[i,:,:]))

In [ ]:
# train and test with classifier
classifier = SVC(verbose = 2)
classifier.fit(hog_train_data,train_labels)

print(classification_report(classifier.predict(hog_test_data), test_labels))

## Question 7

In [ ]:
# this converts idx3-type files to numpy arrays

with open('/content/drive/MyDrive/mnist/MNIST/raw/train-images-idx3-ubyte','rb') as f:
    magic, size = struct.unpack(">II", f.read(8))
    nrows, ncols = struct.unpack(">II", f.read(8))
    train_data = np.fromfile(f, dtype=np.dtype(np.uint8).newbyteorder('>'))
    train_data = train_data.reshape((size, nrows, ncols))

with open('/content/drive/MyDrive/mnist/MNIST/raw/train-labels-idx1-ubyte', 'rb') as i:
    magic, size = struct.unpack('>II', i.read(8))
    train_labels = np.fromfile(i, dtype=np.dtype(np.uint8)).newbyteorder(">")

# ## test images
with open('/content/drive/MyDrive/mnist/MNIST/raw/t10k-images-idx3-ubyte','rb') as f1:
    magic, size = struct.unpack(">II", f1.read(8))
    nrows, ncols = struct.unpack(">II", f1.read(8))
    test_data = np.fromfile(f1, dtype=np.dtype(np.uint8).newbyteorder('>'))
    test_data = test_data.reshape((size, nrows, ncols))

with open('/content/drive/MyDrive/mnist/MNIST/raw/t10k-labels-idx1-ubyte', 'rb') as i:
    magic, size = struct.unpack('>II', i.read(8))
    test_labels = np.fromfile(i, dtype=np.dtype(np.uint8)).newbyteorder(">")

In [ ]:
# get HOG features based on the following parameters
def get_hog_features(j):
  return hog(j, cells_per_block = (2,2), pixels_per_cell = (8, 8))

# iterate over each image and save in numpy array

hog_train_data = []
hog_test_data = []

for i in range(train_data.shape[0]):
  hog_train_data.append(get_hog_features(train_data[i,:,:]))

for i in range(test_data.shape[0]):
  hog_test_data.append(get_hog_features(test_data[i,:,:]))

model_train_data = np.asarray(hog_train_data)
model_test_data = np.asarray(hog_test_data)

In [ ]:
# train a simple SVC classifier and fit the training data
classifier = SVC(verbose = 2)
classifier.fit(model_train_data,train_labels)

# predict from test data and evaluate the results
test_labels_predicted = classifier.predict(model_test_data)
print(classification_report(test_labels, test_labels_predicted))

[LibSVM]              precision    recall  f1-score   support

           0       0.97      0.99      0.98       980
           1       0.99      0.99      0.99      1135
           2       0.97      0.98      0.97      1032
           3       0.96      0.97      0.97      1010
           4       0.97      0.97      0.97       982
           5       0.98      0.97      0.98       892
           6       0.98      0.98      0.98       958
           7       0.98      0.95      0.97      1028
           8       0.96      0.96      0.96       974
           9       0.96      0.96      0.96      1009

    accuracy                           0.97     10000
   macro avg       0.97      0.97      0.97     10000
weighted avg       0.97      0.97      0.97     10000

